# CWT & STFT + CNN Model with BCI_competitionIII dataset

In [1]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from mne.channels import make_standard_montage

raw_each = [0] * 5

for i in range(0,5):
    raw_each[i] = mne.io.read_raw_edf("C:\git\Senior_Thesis\DataSet\Convert_data\hand"+ str(i) +"_new.edf",preload = True)

# raw_edf = mne.concatenate_raws(raw_each)
raw_edf = mne.concatenate_raws([raw_each[3]])
# raw_edf = raw_edf.copy().resample(62.5 ,npad="auto")

eegbci.standardize(raw_edf)  # set channel names
montage = make_standard_montage("standard_1005")
raw_edf.set_montage(montage)

Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand0_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand1_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand2_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand3_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand4_n

<RawEDF | hand3_new.edf, 8 x 279250 (1117.0 s), ~17.1 MB, data loaded>

In [2]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, BatchNormalization, Activation
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import History 
history = History()
import pywt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mne.decoding import CSP
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit,StratifiedKFold ,cross_val_score, cross_val_predict
from ssqueezepy import ssq_cwt, ssq_stft

In [3]:
eeg1 = raw_edf.copy().filter(l_freq=1.0, h_freq=30.0, method = 'iir', iir_params= {"order": 5, "ftype":'butter'})
# eeg1 = raw_edf.copy().filter(l_freq=0.075, h_freq=3.0, method = 'fir')
eeg1 = eeg1.copy().set_eeg_reference(ref_channels="average")

eeg1= eeg1.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','PO8'])
# eeg1= eeg1.pick_channels(["C3", "Cz", "C4"])
events, event_dict = mne.events_from_annotations(eeg1)
combine_epochs = mne.Epochs(eeg1, events, 
        tmin=-5.0,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax=5.0,    # final timestamp (10 means set epoch duration 10 second)
        event_id=event_dict,
        preload = True,
        event_repeated='drop'
    )

combine_epochs = combine_epochs.copy().crop(tmin=0.0, tmax=4.0)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 1.00, 30.00 Hz: -6.02, -6.02 dB



EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_End_Of_Session', 'OVTK_GDF_End_Of_Trial', 'OVTK_GDF_Feedback_Continuous', 'OVTK_GDF_Incorrect', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Start_Of_Trial', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up', 'OVTK_StimulationId_BaselineStart', 'OVTK_StimulationId_BaselineStop', 'OVTK_StimulationId_Beep', 'OVTK_StimulationId_Train']
Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Not setting metadata
605 matching events found
Setting baseline interval to [-5.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 605 events and 2501 original time points ...
2 bad epochs dropped


In [4]:
combine_epochs

Number of events,603
Events,OVTK_GDF_Cross_On_Screen: 1OVTK_GDF_End_Of_Session: 1OVTK_GDF_End_Of_Trial: 120OVTK_GDF_Feedback_Continuous: 120OVTK_GDF_Left: 30OVTK_GDF_Right: 30OVTK_GDF_Start_Of_Trial: 120OVTK_GDF_Tongue: 30OVTK_GDF_Up: 30OVTK_StimulationId_BaselineStart: 0OVTK_StimulationId_BaselineStop: 1OVTK_StimulationId_Beep: 120OVTK_StimulationId_Train: 0
Time range,0.000 – 4.000 s
Baseline,-5.000 – 0.000 s


In [5]:
combine_epochs['OVTK_GDF_Left','OVTK_GDF_Right','OVTK_GDF_Up','OVTK_GDF_Tongue'].get_data().shape[2]
labels = combine_epochs['OVTK_GDF_Left','OVTK_GDF_Right','OVTK_GDF_Up','OVTK_GDF_Tongue'].events[:,2]

In [6]:
for i in range(0,len(labels)):
    if labels[i] > 7:
        labels[i] = labels[i] - 1

shape = np.shape(combine_epochs['OVTK_GDF_Left','OVTK_GDF_Right','OVTK_GDF_Up','OVTK_GDF_Tongue'].get_data())
train_data = combine_epochs['OVTK_GDF_Left','OVTK_GDF_Right','OVTK_GDF_Up','OVTK_GDF_Tongue'].get_data()

In [7]:
np.shape(train_data)

(120, 7, 1001)

In [8]:
from mne.decoding import CSP
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=7, reg=None, log=None, norm_trace=False, transform_into='csp_space')
# clf = Pipeline([("CSP", csp), ("LDA", lda)])
# scores = cross_val_score(clf, train_data, labels, cv=5, n_jobs=None)

csp.fit(train_data, labels)
new_data = csp.transform(train_data)

# array_2d = csp_coeff.reshape(csp_coeff.shape[0], csp_coeff.shape[1]*csp_coeff.shape[2])
# x_train, x_test, y_train, y_test = train_test_split(array_2d, labels, test_size=0.2, random_state=42)
# lda.fit(x_train,y_train)
# scores = lda.score(x_test, y_test)
# print(scores)

Computing rank from data with rank=None
    Using tolerance 1.9 (2.2e-16 eps * 7 dim * 1.2e+15  max singular value)
    Estimated rank (mag): 7
    MAG: rank 7 computed from 7 data channels with 0 projectors
Reducing data rank from 7 -> 7
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.9 (2.2e-16 eps * 7 dim * 1.2e+15  max singular value)
    Estimated rank (mag): 7
    MAG: rank 7 computed from 7 data channels with 0 projectors
Reducing data rank from 7 -> 7
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.9 (2.2e-16 eps * 7 dim * 1.2e+15  max singular value)
    Estimated rank (mag): 7
    MAG: rank 7 computed from 7 data channels with 0 projectors
Reducing data rank from 7 -> 7
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.1 (2.2e-16 eps * 7 dim * 1.3e+15  max singular value)
    Estimated rank (mag): 7
    MAG

In [9]:
np.shape(new_data)

(120, 7, 1001)

In [10]:
# from ssqueezepy import ssq_cwt, ssq_stft

# n_fft = 256  # Number of DFT points
# hop_length = int(n_fft * 0.03)  # 97% overlapping
# win_length = int(n_fft * 0.5)   # 0.5 seconds window length
# window = 'hamming' 
# train_size = len(labels)
# train_data_stft = np.ndarray(shape=(train_size, 129,143,7))


# _,coeff, *_ = ssq_stft(new_data[0,:], n_fft=n_fft, hop_len=hop_length, win_len=win_length, window=window)
# for i in range(0,train_size):
#     _,coeff, *_ = ssq_stft(new_data[0,:], n_fft=n_fft, hop_len=hop_length, win_len=win_length, window=window)
#     train_data_stft[i, :, :, :] = abs(coeff.reshape(coeff.shape[1],coeff.shape[2],coeff.shape[0]))

# np.shape(train_data_stft)

In [13]:
# import numpy as np
# import pywt
# import matplotlib.pyplot as plt

# # Sample image dimensions
# def to_image(coeff):
#     width = 37
#     height = 7

#     # Generate a sample 2D CWT array (replace this with your actual CWT coefficients)
#     cwt_coefficients = abs(coeff)

#     # Create an empty 3D array for the image (assuming RGB color channels)
#     color_channels = 3
#     image_3d = np.zeros((width, height, color_channels), dtype=np.uint8)

#     # Map the CWT coefficients to color channels for visualization
#     # You may need to adjust this mapping based on your specific data
#     min_value = np.min(cwt_coefficients)
#     max_value = np.max(cwt_coefficients)
#     normalized_cwt = (cwt_coefficients - min_value) / (max_value - min_value)

#     # Assign the CWT coefficients to the color channels
#     image_3d[:, :, 0] = (normalized_cwt * 255).astype(np.uint8)  # Red channel
#     image_3d[:, :, 1] = (normalized_cwt * 255).astype(np.uint8)  # Green channel
#     image_3d[:, :, 2] = (normalized_cwt * 255).astype(np.uint8)  # Blue channel

#     return image_3d

In [14]:
np.shape(new_data)

(120, 7, 1001)

In [15]:
train_cwt = np.ndarray(shape=(120, 1001, 7))
for jj in range(0, new_data.shape[0]):
    train_cwt[jj] = new_data[jj].T
print(np.shape(new_data))

scales = range(1,31)

waveletname = 'morl'
train_size = len(labels)
train_data_cwt = np.ndarray(shape=(train_size, 30, 1001, 7))

for ii in range(0,train_size):
    if ii % 40 == 0:
        print(ii)
    for jj in range(0,7):
        signal = train_cwt[ii, :, jj]
        coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:1001]  #crop 227 sample for each channel
        train_data_cwt[ii, :, :, jj] = np.abs(coeff_)
print(np.shape(coeff),np.shape(coeff_))

(120, 7, 1001)
0


40
80
(30, 1001) (30, 1001)


In [16]:
x_train, x_test, y_train, y_test = train_test_split(train_data_cwt, labels, test_size=0.2, random_state=42)

img_x = train_data_cwt.shape[1]
img_y = train_data_cwt.shape[2]
img_z = train_data_cwt.shape[3]
num_classes = 4

batch_size = 12
epochs = 100
print(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

# del train_data_cwt
# del combine_epochs
# plt.imshow(x_train[0,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
# plt.imshow(x_train[2,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
y_train = keras.utils.to_categorical(y_train - 6, num_classes)
y_test = keras.utils.to_categorical(y_test -6 , num_classes)
input_shape = (img_x, img_y, img_z)

(96, 30, 1001, 7) (24, 30, 1001, 7) (96,) (24,)


In [17]:
# Define the model
model = Sequential()

# Input Layer (I1)
model.add(Input(shape=input_shape))

# Convolution Layer (C2)
model.add(Conv2D(8, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Convolution Layer (C3)
model.add(Conv2D(16, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Flatten Layer (F4)
model.add(Flatten())

# Fully Connected Layer (D5)
model.add(Dense(64))

# Output Layer (O6)
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(learning_rate= 0.001), 
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, 
          epochs=epochs, verbose=1, 
          validation_data=(x_test, y_test), 
          callbacks=[history])

Epoch 1/100
8/8 [==============================] - 5s 521ms/step - loss: 572.8930 - accuracy: 0.2604 - val_loss: 483.4688 - val_accuracy: 0.2083
Epoch 2/100
8/8 [==============================] - 4s 478ms/step - loss: 135.0417 - accuracy: 0.5833 - val_loss: 254.2366 - val_accuracy: 0.2917
Epoch 3/100
8/8 [==============================] - 4s 513ms/step - loss: 16.3045 - accuracy: 0.9062 - val_loss: 139.9712 - val_accuracy: 0.4583
Epoch 4/100
8/8 [==============================] - 4s 466ms/step - loss: 14.8379 - accuracy: 0.9271 - val_loss: 98.8976 - val_accuracy: 0.4167
Epoch 5/100
8/8 [==============================] - 4s 499ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 255.1499 - val_accuracy: 0.3333
Epoch 6/100
8/8 [==============================] - 4s 487ms/step - loss: 1.1578 - accuracy: 0.9896 - val_loss: 183.6643 - val_accuracy: 0.3333
Epoch 7/100
8/8 [==============================] - 4s 484ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 133.4041 - val_acc

KeyboardInterrupt: 

In [ ]:
# model = Sequential()
# # Layer 1: Convolutional + Max-Pooling
# model.add(Conv2D(filters= 96, kernel_size= (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
# model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# # Layer 2: Convolutional + Max-Pooling
# model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1) ,activation='relu', padding= 'valid'))
# model.add(MaxPooling2D((3, 3), strides=(2, 2), padding= 'valid'))

# # Layer 3: Convolutional
# model.add(Conv2D(filters =384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# # Layer 4: Convolutional
# model.add(Conv2D(filters = 384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# # Layer 5: Convolutional + Max-Pooling
# model.add(Conv2D(filters = 256, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))
# model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='valid'))

# # Flatten the output for fully connected layers
# model.add(Flatten())

# # Layer 6: Fully Connected (Dense)
# model.add(Dense(4096, activation='relu', input_shape = (13,13,128)))
# model.add(keras.layers.Dropout(0.4))

# # Layer 7: Fully Connected (Dense)
# model.add(Dense(2048, activation='relu'))
# model.add(keras.layers.Dropout(0.4))

# # Layer 8: Output Layer
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy, 
#               optimizer=keras.optimizers.Adam(learning_rate= 0.0001), 
#               metrics=['accuracy'])

# model.fit(x_train, y_train, batch_size=batch_size, 
#           epochs=epochs, verbose=1, 
#           validation_data=(x_test, y_test), 
#           callbacks=[history])

Epoch 1/100
8/8 [==============================] - 18s 2s/step - loss: 1.4239 - accuracy: 0.2083 - val_loss: 1.4117 - val_accuracy: 0.1667
Epoch 2/100
8/8 [==============================] - 15s 2s/step - loss: 1.3922 - accuracy: 0.2812 - val_loss: 1.3924 - val_accuracy: 0.1667
Epoch 3/100
8/8 [==============================] - 15s 2s/step - loss: 1.3866 - accuracy: 0.2292 - val_loss: 1.3951 - val_accuracy: 0.2083
Epoch 4/100
8/8 [==============================] - 15s 2s/step - loss: 1.3565 - accuracy: 0.2604 - val_loss: 1.3802 - val_accuracy: 0.2917
Epoch 5/100
8/8 [==============================] - 15s 2s/step - loss: 1.3101 - accuracy: 0.3333 - val_loss: 1.5798 - val_accuracy: 0.1667
Epoch 6/100
8/8 [==============================] - 15s 2s/step - loss: 1.1766 - accuracy: 0.5104 - val_loss: 1.0649 - val_accuracy: 0.5417
Epoch 7/100
8/8 [==============================] - 15s 2s/step - loss: 1.1400 - accuracy: 0.4583 - val_loss: 1.0173 - val_accuracy: 0.5833
Epoch 8/100
8/8 [==========

KeyboardInterrupt: 

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Train loss: 0.0, Train accuracy: 1.0
Test loss: 76.14993286132812, Test accuracy: 0.6666666865348816
